<a href="https://colab.research.google.com/github/kimjeongsoo20190147/dacon/blob/main/xg%2Brf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch.utils.data import Dataset, DataLoader
import transformers
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm
import pandas as pd
from types import SimpleNamespace
from sklearn.preprocessing import LabelEncoder
from transformers import BertModel
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from collections import Counter

In [4]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [5]:
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash
!sudo apt-get install -y mecab mecab-ipadic-utf8 libmecab-dev swig
!git clone https://bitbucket.org/eunjeon/mecab-ko-dic.git
!bash mecab-ko-dic/tools/install.sh
!pip install mecab-python3
!wget https://bitbucket.org/eunjeon/mecab-ko-dic/downloads/mecab-ko-dic-2.1.1-20180720.tar.gz
!tar xvfz mecab-ko-dic-2.1.1-20180720.tar.gz
!cd mecab-ko-dic-2.1.1-20180720
!./configure
!make
!sudo make install

--2024-10-17 12:05:17--  https://www.dropbox.com/s/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/g8l6dlu5a3ebovcdymluq/mecab-0.996-ko-0.9.2.tar.gz?rlkey=xlooryouhr1l2ufqbt93ohfvy&dl=1 [following]
--2024-10-17 12:05:17--  https://www.dropbox.com/scl/fi/g8l6dlu5a3ebovcdymluq/mecab-0.996-ko-0.9.2.tar.gz?rlkey=xlooryouhr1l2ufqbt93ohfvy&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc4ec6294e0741dde3a44d10842.dl.dropboxusercontent.com/cd/0/inline/CcnDsIyxtaV9DKpzt4LwAn_8eHgV4--BxfkqUyjW0oBjA2yo5yEKasuH6A-LJwHbCYTT6CRxjJz1_XmbJHxGMmiPYqzJ1ybpItlzgXSZK41EV8h8lNstAWpGpJGyRfwlKWblO3CNqvUP7tRJW9-hRHvB/file?dl=1# [following]
--2024-10-17 12:05:18--  https://u

In [6]:
from sklearn.model_selection import train_test_split

X_train = train_df.drop(['분류'], axis=1)
y_train = train_df["분류"]
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, stratify=y_train, test_size=0.1, random_state=8678686)

# # 테스트데이터 따로 추출
# _, X_test, _, y_test = train_test_split(X_train, y_train,stratify = y_train, test_size=0.2, random_state=8678686)

# # 트레인데이터 따로 추출
# X_train, _, y_train, _ = train_test_split(X_train, y_train,
#                                           test_size=0.8,
#                                           stratify=y_train,
#                                           random_state=42)

#Encoded Number
print(y_train)

label_encoder = LabelEncoder()

y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

print(y_train_encoded)
print(y_test_encoded)

# print(f"전체 데이터 크기: {len(X_train)}")
# print(f"샘플링된 데이터 크기: {len(X_sampled)}")

X_real_test = test_df

19053     경제:산업_기업
9046     경제:금융_재테크
12898    IT_과학:모바일
4352            지역
45757     사회:사건_사고
           ...    
51724       경제:자동차
49103           지역
22464       사회:장애인
1670            지역
37511           지역
Name: 분류, Length: 49148, dtype: object
[11  7  2 ... 39 55 55]
[55 55 38 ... 45 55 11]


In [7]:
from konlpy.tag import Mecab
import konlpy

# 제목 데이터만 토큰화
print(X_train['제목'])

19053               조양호 5주기 ‘수송보국’ 일대기 담은 평전 출간
9046           용인시, 학자금 대출 청년 신용 회복 지원 최대 100만원
12898               백문이 불여일견 기아, 인증중고차 실물 상담 실시
4352             용인시, 산업단지 조성원가 분양수익 확정절차 개선 시행
45757               '탄핵 추진' 검사 압수수색 '이재명 수사' 배제
                          ...                  
51724       르노삼성, 최성규 신임 연구소장 임명 “친환경 신차 개발 주도”
49103              용인 김포 구리시 '미세먼지 저감' 최우수기관 선정
22464         용인시, 공공기관에 ‘장애인 안내견 환영’ 점자 안내판 부착
1670               용인시, 국민권익위원회 청렴도 평가 3등급 ‘추락’
37511    기흥호수공원 골프연습장 잇따른 논란  백군기 용인시장은 그동안 뭐했나
Name: 제목, Length: 49148, dtype: object


In [8]:
mecab = Mecab(dicpath='/usr/local/lib/mecab/dic/mecab-ko-dic')

# 제목 데이터만 토큰화

X_train['제목'] = [(mecab.morphs(text)) for text in X_train['제목']]
X_test['제목'] = [mecab.morphs(text) for text in X_test['제목']]
X_real_test['제목'] = [mecab.morphs(text) for text in X_real_test['제목']]
print("분석완료\n",X_train['제목'])


분석완료
 19053        [조양호, 5, 주기, ‘, 수송, 보국, ’, 일대기, 담, 은, 평전, 출간]
9046      [용인시, ,, 학자금, 대출, 청년, 신용, 회복, 지원, 최대, 100, 만, 원]
12898            [백문, 이, 불여일견, 기아, ,, 인증, 중고차, 실물, 상담, 실시]
4352        [용인시, ,, 산업단지, 조성, 원가, 분양, 수익, 확정, 절차, 개선, 시행]
45757          [', 탄핵, 추진, ', 검사, 압수수색, ', 이재명, 수사, ', 배제]
                               ...                        
51724    [르노삼성, ,, 최성규, 신임, 연구, 소장, 임명, “, 친환경, 신차, 개발,...
49103           [용인, 김포, 구리시, ', 미세먼지, 저감, ', 최우수, 기관, 선정]
22464    [용인시, ,, 공공, 기관, 에, ‘, 장애, 인, 안내견, 환영, ’, 점자, ...
1670       [용인시, ,, 국민, 권익, 위원회, 청렴도, 평가, 3, 등급, ‘, 추락, ’]
37511    [기흥, 호수, 공원, 골프, 연습장, 잇따른, 논란, 백군기, 용인, 시장, 은,...
Name: 제목, Length: 49148, dtype: object


In [18]:
stop_words = pd.read_csv('stop_words.csv')
print("불용어 확인",stop_words)

X_train['제목'] = [[word for word in sen if not word in stop_words ] for sen in X_train['제목']]
X_test['제목']  =  [[word for word in sen if not word in stop_words ] for sen in X_test['제목']]
X_real_test['제목'] =  [[word for word in sen if not word in stop_words ] for sen in X_real_test['제목']]

불용어 확인   Column1 Column2 Column3 Column4 Column5 Column6 Column7 Column8 Column9  \
0       어       나      우리      저희      따라      의해       의       을       를   

  Column10  ... Column31 Column32 Column33 Column34 Column35 Column36  \
0        에  ...        과        “        ,        면        ”        ‘   

  Column37 Column38 Column39 Column40  
0        ’        .       ..        …  

[1 rows x 40 columns]


In [19]:
X_train['키워드'] = X_train['키워드'].apply(lambda x: x.split(','))
X_test['키워드'] = X_test['키워드'].apply(lambda x: x.split(','))
X_real_test['키워드'] = X_real_test['키워드'].apply(lambda x: x.split(','))

X_train['combined_text'] = X_train['제목'] + X_train['키워드']
X_test['combined_text'] = X_test['제목'] + X_test['키워드']
X_real_test['combined_text'] = X_real_test['제목'] + X_real_test['키워드']

print(X_train.head(3))

                ID                                                제목  \
19053  TRAIN_19053     [조양호, 5, 주기, ‘, 수송, 보국, ’, 일대기, 담, 은, 평전, 출간]   
9046   TRAIN_09046  [용인시, ,, 학자금, 대출, 청년, 신용, 회복, 지원, 최대, 100, 만, 원]   
12898  TRAIN_12898         [백문, 이, 불여일견, 기아, ,, 인증, 중고차, 실물, 상담, 실시]   

                                                     키워드  \
19053  [조양호, 수송보국, 일대기, 평전, 출간, 추모제, 공개, 챕터, 변화, 재조명,...   
9046   [용인시, 학자금, 대출, 청년, 신용, 회복, 지원, 최대, 100만, 노호근, ...   
12898  [백문, 불여일견, 기아, 상담, 인증, 중고차, 실물, 상담, 아시아투데이, 강태...   

                                           combined_text  
19053  [조양호, 5, 주기, ‘, 수송, 보국, ’, 일대기, 담, 은, 평전, 출간, ...  
9046   [용인시, ,, 학자금, 대출, 청년, 신용, 회복, 지원, 최대, 100, 만, ...  
12898  [백문, 이, 불여일견, 기아, ,, 인증, 중고차, 실물, 상담, 실시, 백문, ...  


In [20]:
print(X_train["combined_text"])

19053    [조양호, 5, 주기, ‘, 수송, 보국, ’, 일대기, 담, 은, 평전, 출간, ...
9046     [용인시, ,, 학자금, 대출, 청년, 신용, 회복, 지원, 최대, 100, 만, ...
12898    [백문, 이, 불여일견, 기아, ,, 인증, 중고차, 실물, 상담, 실시, 백문, ...
4352     [용인시, ,, 산업단지, 조성, 원가, 분양, 수익, 확정, 절차, 개선, 시행,...
45757    [', 탄핵, 추진, ', 검사, 압수수색, ', 이재명, 수사, ', 배제, 탄핵...
                               ...                        
51724    [르노삼성, ,, 최성규, 신임, 연구, 소장, 임명, “, 친환경, 신차, 개발,...
49103    [용인, 김포, 구리시, ', 미세먼지, 저감, ', 최우수, 기관, 선정, 용인,...
22464    [용인시, ,, 공공, 기관, 에, ‘, 장애, 인, 안내견, 환영, ’, 점자, ...
1670     [용인시, ,, 국민, 권익, 위원회, 청렴도, 평가, 3, 등급, ‘, 추락, ’...
37511    [기흥, 호수, 공원, 골프, 연습장, 잇따른, 논란, 백군기, 용인, 시장, 은,...
Name: combined_text, Length: 49148, dtype: object


In [21]:
word2vec_model = Word2Vec(sentences=X_train['combined_text'], vector_size=230, window=5, min_count=1, workers=4)

def sentence_vector(sentence, model):
    word_vecs = [model.wv[word] for word in sentence if word in model.wv]
    return np.mean(word_vecs, axis=0) if word_vecs else np.zeros(model.vector_size)

X_train_word2vec_both = np.array([sentence_vector(sentence, word2vec_model) for sentence in X_train['combined_text']])
X_test_word2vec_both = np.array([sentence_vector(sentence, word2vec_model) for sentence in X_test['combined_text']])
X_real_test_word2vec_both = np.array([sentence_vector(sentence, word2vec_model) for sentence in X_real_test['combined_text']])


In [22]:
from imblearn.over_sampling import ADASYN
print(Counter(y_train_encoded).items())

# Calculate 50% of the most frequent class size
max_class_size = max(Counter(y_train_encoded).values())
target_ratio = 0.5 * max_class_size

# Set sampling strategy to limit oversampling to 50% of the largest class
adasyn = ADASYN(sampling_strategy={cls: target_ratio for cls, count in Counter(y_train_encoded).items() if count < target_ratio}, random_state=2024)


# adasyn = ADASYN(random_state=2024)
X_train_resampled, y_train_resampled = adasyn.fit_resample(X_train_word2vec_both, y_train_encoded)

print(Counter(y_train_resampled).items())

dict_items([(11, 640), (7, 294), (2, 483), (55, 24255), (35, 2311), (38, 855), (9, 2086), (10, 3109), (48, 869), (30, 182), (39, 559), (41, 555), (47, 295), (5, 144), (40, 356), (36, 1332), (8, 338), (29, 223), (6, 563), (33, 402), (50, 547), (52, 199), (45, 757), (15, 576), (32, 112), (18, 761), (43, 26), (27, 604), (34, 115), (54, 314), (37, 482), (4, 214), (31, 896), (14, 530), (16, 160), (21, 206), (12, 348), (25, 171), (0, 219), (53, 251), (46, 81), (1, 194), (23, 92), (19, 303), (13, 8), (28, 156), (42, 102), (20, 192), (17, 68), (22, 302), (26, 99), (49, 60), (24, 24), (51, 28), (3, 85), (44, 15)])
dict_items([(11, 12127), (7, 12153), (2, 12143), (55, 24255), (35, 12269), (38, 12124), (9, 12016), (10, 12113), (48, 12064), (30, 12130), (39, 12140), (41, 12103), (47, 12125), (5, 12119), (40, 12074), (36, 12148), (8, 12127), (29, 12073), (6, 12254), (33, 12159), (50, 12117), (52, 12120), (45, 12224), (15, 12177), (32, 12133), (18, 12169), (43, 12127), (27, 12120), (34, 12148), (54,

In [23]:
!pip install scikit-learn

In [24]:
len(X_train_word2vec_both)

49148

In [25]:
# Train Random Forest model
#with subsample
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import StackingClassifier, BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import xgboost as xgb
import numpy as np

rf_best_model = RandomForestClassifier(n_estimators=200,
                                        random_state=2024,
                                        min_samples_leaf=2,
                                        min_samples_split=4)

xgb_best_model = xgb.XGBClassifier(n_estimators=200,
                                   random_state=2024,
                                   min_child_weight=2,
                                   max_depth=6)

stacking_clf = StackingClassifier(
    estimators=[
        ('rf', rf_best_model),
        ('xgb', xgb_best_model)
    ],
    final_estimator=RandomForestClassifier(n_estimators=100,
                                           random_state=2024), cv=5
)

#여긴 본 데이터 양만큼 1번 학습 시킬 것임
def train_stacking_model(x_train,y_train,x_test):
    stacking_clf.fit(x_train, y_train)

    y_pred = stacking_clf.predict(x_test)
    y_pred_prob = stacking_clf.predict_proba(x_test)

    return (y_pred, y_pred_prob)


y_pred, y_pred_prob = train_stacking_model(X_train_resampled,y_train_resampled, X_test_word2vec_both)

In [28]:
# rf_model_accuracy = accuracy_score(y_test_encoded,y_pred)
# rf_model_precision = precision_score(y_test_encoded,y_pred,average = 'weighted')
# rf_model_recall = recall_score(y_test_encoded,y_pred,average = 'weighted')
# rf_model_f1 = f1_score(y_test_encoded,y_pred,average = 'weighted')


# # Training set predictions
y_train_pred = stacking_clf.predict(X_train_word2vec_both)

# Test set predictions (you might have this from your previous code)
y_test_pred = stacking_clf.predict(X_test_word2vec_both)

# Test set predictions (you might have this from your previous code)

f1_train = f1_score(y_train_encoded, y_train_pred, average='weighted')
f1_test = f1_score(y_test_encoded, y_test_pred, average='weighted')

print(f"F1 Score - Training Set: {f1_train}")
print(f"F1 Score - Test Set: {f1_test}")

F1 Score - Training Set: 0.9998376941880682
F1 Score - Test Set: 0.790052027226357


In [30]:
# 2. 예측 수행
y_test_pred = stacking_clf.predict(X_real_test_word2vec_both)

# 3. 결과 저장: ID와 예측된 분류 결과를 DataFrame으로 변환
test_results = pd.DataFrame({
    'ID': X_real_test["ID"],    # 테스트 데이터의 ID
    '분류': label_encoder.inverse_transform(y_test_pred)     # 예측된 분류 값
})

# 4. 결과를 CSV 파일로 저장
test_results.to_csv('submission_vol.csv', index=False)


# CSV 결과 예시
print(test_results.head())

           ID        분류
0  TEST_00000   정치:정치일반
1  TEST_00001   사회:사회일반
2  TEST_00002   정치:정치일반
3  TEST_00003  경제:취업_창업
4  TEST_00004        지역
